In [1]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("ToyotaCorolla.csv", encoding="utf-8")

# 데이터 확인
print(df.head())
print(df.info())



   Id                                          Model  Price  Age_08_04     KM  \
0   1  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13500         23  46986   
1   2  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13750         23  72937   
2   3  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13950         24  41711   
3   4  TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  14950         26  48000   
4   5    TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors  13750         30  38500   

   HP    CC  Doors  Cylinders  Quarterly_Tax  ...  Powered_Windows  \
0  90  2000      3          4            210  ...                1   
1  90  2000      3          4            210  ...                0   
2  90  2000      3          4            210  ...                0   
3  90  2000      3          4            210  ...                0   
4  90  2000      3          4            210  ...                1   

   Power_Steering  Radio  Mistlamps  Sport_Model  Backseat_Divider  \
0               1     

In [2]:
# 데이터 열 확인
print(df.columns)

Index(['Id', 'Model', 'Price', 'Age_08_04', 'KM', 'HP', 'CC', 'Doors',
       'Cylinders', 'Quarterly_Tax', 'Weight', 'ABS', 'Airbag_1', 'Airbag_2',
       'Airco', 'Automatic_airco', 'Boardcomputer', 'CD_Player',
       'Central_Lock', 'Powered_Windows', 'Power_Steering', 'Radio',
       'Mistlamps', 'Sport_Model', 'Backseat_Divider', 'Metallic_Rim',
       'Radio_cassette', 'Parking_Assistant', 'Tow_Bar'],
      dtype='object')


In [3]:
import statsmodels.api as sm


# 사용하지 않을 열 삭제 (Id, Model 등 비필요한 열)
df = df.drop(columns=['Id', 'Model'])

# 결측값 확인 및 제거 (필요시)
df = df.dropna()


In [4]:

# 독립변수(X)와 종속변수(y) 정의
X = df.drop(columns=['Price'])  # 'Price'를 제외한 나머지 변수
y = df['Price']  # 타겟 변수

# 상수항 추가 (절편)
X = sm.add_constant(X, has_constant = "add")

# 다중회귀모델 구축
model = sm.OLS(y, X).fit()

# 결과 출력
print(model.summary())



                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     469.5
Date:                Sun, 23 Mar 2025   Prob (F-statistic):               0.00
Time:                        23:34:56   Log-Likelihood:                -12204.
No. Observations:                1436   AIC:                         2.446e+04
Df Residuals:                    1410   BIC:                         2.460e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               254.3040     64.27

In [5]:
# P-value가 0.05 이상인 변수 필터링
insignificant_vars = model.pvalues[model.pvalues >= 0.05]

# 결과 출력
print("P-value가 0.05 이상인 변수들:")
print(insignificant_vars)

P-value가 0.05 이상인 변수들:
Airbag_1             0.305254
Airbag_2             0.277778
Airco                0.091175
Boardcomputer        0.061557
Central_Lock         0.757268
Power_Steering       0.744135
Radio                0.318191
Mistlamps            0.663109
Backseat_Divider     0.342570
Metallic_Rim         0.106635
Radio_cassette       0.299905
Parking_Assistant    0.316276
dtype: float64


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# 데이터를 80% Train / 20% Test로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 회귀 모델 적합 (Training set 사용)
model = sm.OLS(y_train, X_train).fit()

# Testing set에서 예측값 구하기
y_pred = model.predict(X_test)

# 예측 정확도(R²) 계산
r2 = r2_score(y_test, y_pred)

print(f"Testing set의 예측 정확도 (R²): {r2:.4f}")

Testing set의 예측 정확도 (R²): 0.8902


In [7]:
# 2. P-value가 0.05 이상인 변수들 제거
X_train_filtered = X_train.drop(columns=insignificant_vars.index)
X_test_filtered = X_test.drop(columns=insignificant_vars.index)

# 3. 모델 훈련 (변수 제거 후)
model_filtered = sm.OLS(y_train, X_train_filtered).fit()

# 4. 모델 평가 (R² 값 계산)
y_pred_filtered = model_filtered.predict(X_test_filtered)
r2_filtered = r2_score(y_test, y_pred_filtered)
print(f"변수 제거 후 모델 성능 (R²): {r2_filtered:.4f}")

# 5. 원래 모델 평가 (R² 값 계산)
print(f"원래 모델 성능 (R²): {r2:.4f}")

# 6. 성능 차이 확인
if r2_filtered > r2:
    print("변수 제거 후 모델 성능이 향상되었습니다.")
else:
    print("변수 제거 후 모델 성능이 저하되었습니다.")

변수 제거 후 모델 성능 (R²): 0.8929
원래 모델 성능 (R²): 0.8902
변수 제거 후 모델 성능이 향상되었습니다.
